In [5]:
 # for database connections
import sqlalchemy
from sqlalchemy import create_engine, inspect, text

# data
import pandas as pd
import numpy as np

# viz
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
SQL_USERNAME = "postgres"
SQL_PASSWORD = "postgres" # unique to user
SQL_IP = "localhost"
PORT = 5432
DATABASE = "ETL_MiniProject" # unique to user

In [7]:
connection_string = f"postgresql+psycopg2://{SQL_USERNAME}:{SQL_PASSWORD}@{SQL_IP}:{PORT}/{DATABASE}"
engine = create_engine(connection_string) 

In [10]:
connection = engine.connect()

In [13]:
# TEST 
# Query the "campaign" table and load the results into a DataFrame
query = "SELECT * FROM campaign;"
campaign_data = pd.read_sql(query, connection)

# Display the data in the DataFrame
campaign_data

,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launched_date,end_date,category_id,subcategory_id
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100,0,failed,0,CA,CAD,2020-02-13,2021-03-01,cat1,subcat1
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400,14560,successful,158,US,USD,2021-01-25,2021-05-25,cat2,subcat2
2,1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400,142523,successful,1425,AU,AUD,2020-12-17,2021-12-30,cat3,subcat3
3,2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200,2477,failed,24,US,USD,2021-10-21,2022-01-17,cat2,subcat2
4,1365,2199,Larson-Little,Proactive foreground core,7600,5265,failed,53,US,USD,2020-12-21,2021-08-23,cat4,subcat4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2986,3684,Manning-Hamilton,Vision-oriented scalable definition,97300,153216,successful,2043,US,USD,2020-12-29,2021-05-30,cat1,subcat1
996,2031,5784,Butler LLC,Future-proofed upward-trending migration,6600,4814,failed,112,US,USD,2021-10-15,2021-11-30,cat4,subcat4
997,1627,1498,Ball LLC,Right-sized full-range throughput,7600,4603,canceled,139,IT,EUR,2021-11-06,2021-12-10,cat4,subcat4
998,2175,6073,"Taylor, Santiago and Flores",Polarized composite customer loyalty,66600,37823,failed,374,US,USD,2020-10-08,2021-04-11,cat2,subcat8


In [16]:
# Query 1: How many backers, what did they pledge and what was the outcome
query = "SELECT cf_id, backers_count, pledged, outcome FROM campaign;"

# Execute the query and load the results into a DataFrame
outcome_data = pd.read_sql(query, connection)

# Display the data in the DataFrame
outcome_data

,cf_id,backers_count,pledged,outcome
0,147,0,0,failed
1,1621,158,14560,successful
2,1812,1425,142523,successful
3,2156,24,2477,failed
4,1365,53,5265,failed
...,...,...,...,...
995,2986,2043,153216,successful
996,2031,112,4814,failed
997,1627,139,4603,canceled
998,2175,374,37823,failed


In [ ]:
query = "SELECT cf_id, backers_count, pledged, outcome FROM campaign;"

# Execute the query and load the results into a DataFrame
outcome_data = pd.read_sql(query, connection)

# Display the data in the DataFrame
outcome_data